In [1]:
from getpass import getpass
import os
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()
# 兩個 API 的密鑰
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPERDEV_API_KEY"] = os.getenv("SERPERDEV_API_KEY")
# 判斷是否存在，不存在可透過手動輸入
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
if "SERPERDEV_API_KEY" not in os.environ:
    os.environ["SERPERDEV_API_KEY"] = getpass("Enter Serper API key:")

In [2]:
# 用於建立文件
from haystack.dataclasses import Document

# 定義一個文件，內容描述了德國南部巴伐利亞州首府慕尼黑
documents = [
    Document(
        content="""慕尼黑是德國南部巴伐利亞州的首府，充滿了豐富的文化遺產和現代城市的優雅。位於伊薩爾河畔，慕尼黑以其華麗的建築而聞名，如馬麗恩廣場的慕尼黑新市政廳和寧芬堡宮的宏偉壯觀。這座城市是藝術愛好者的天堂，擁有世界級的博物館，如阿爾特·皮納科特克，收藏了許多著名藝術家的傑作。慕尼黑也因其熱鬧的啤酒花園而聞名，當地人和遊客聚集在這裡享受城市著名的啤酒和傳統巴伐利亞美食。每年的慕尼黑啤酒節是世界上最大的啤酒節，吸引了來自世界各地的數百萬遊客。除了豐富的文化和美食享受外，慕尼黑還擁有如英國花園這樣的美麗公園，為繁忙的都市提供了一個寧靜的避風港。遊客們為慕尼黑的熱情好客所打動，使得這裡成為了想要體驗古老魅力和現代魅力的旅行者必去的目的地。"""
    )
]

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator

# 定義一個提示模板，指導 LLM 回答查詢
prompt_template = """
根據文件回答下列問題。
如果答案未包含在文件中，請回覆
'no_answer'

Query: {{query}}

Documents:
{% for document in documents %}
    {{document.content}}
{% endfor %}
"""

# 使用定義的模板建立 PromptBuilder
prompt_builder = PromptBuilder(template=prompt_template)

# 初始化 OpenAIGenerator，使用 gpt-4-turbo 模型
llm = OpenAIGenerator(model="gpt-4-turbo")

In [4]:
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.components.websearch.serper_dev import SerperDevWebSearch

# 定義一個提示模板，用於網絡搜索
prompt_for_websearch = """
根據從 Web 檢索到的文件，回答以下查詢。
你的答案應表明你的答案是透過網路搜尋產生的。

Query: {{query}}
Documents:
{% for document in documents %}
    {{document.content}}
{% endfor %}
"""

# 初始化網絡搜索組件
websearch = SerperDevWebSearch()

# 使用網絡搜索提示模板建立 PromptBuilder
prompt_builder_for_websearch = PromptBuilder(template=prompt_for_websearch)

# 初始化 OpenAIGenerator，使用 gpt-4-turbo 模型
llm_for_websearch = OpenAIGenerator(model="gpt-4-turbo")

In [5]:
from haystack.components.routers import ConditionalRouter

# 定義路由規則
routes = [
    {
        # 如果 LLM 回覆中包含 'no_answer'
        "condition": "{{'no_answer' in replies[0]}}",
        # 將查詢作為輸出
        "output": "{{query}}",
        # 輸出名稱
        "output_name": "go_to_websearch",
        # 輸出類型
        "output_type": str
    },
    {
        # 如果 LLM 回覆中不包含 'no_answer'
        "condition": "{{'no_answer' not in replies[0]}}",
        # 將回覆的答案作為輸出
        "output": "{{replies[0]}}",
        # 輸出名稱
        "output_name": "answer",
        # 輸出類型
        "output_type": str
    }
]

# 初始化 ConditionalRouter
router = ConditionalRouter(routes)

In [6]:
from haystack import Pipeline

# 建立管道
pipe = Pipeline()

# 添加組件到管道
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.add_component("router", router)
pipe.add_component("websearch", websearch)
pipe.add_component("prompt_builder_for_websearch", prompt_builder_for_websearch)
pipe.add_component("llm_for_websearch", llm_for_websearch)

# 連接管道中的組件
# 連接 prompt_builder 到 llm
pipe.connect(
    "prompt_builder", "llm"
)
# 連接 llm 的回覆到 router
pipe.connect(
    "llm.replies", "router.replies"
)
# 連接 router 的 go_to_websearch 到 websearch 的查詢
pipe.connect(
    "router.go_to_websearch", "websearch.query"
)
# 連接 router 的 go_to_websearch 到 prompt_builder_for_websearch 的查詢
pipe.connect(
    "router.go_to_websearch", "prompt_builder_for_websearch.query"
)
# 連接 websearch 的文件到 prompt_builder_for_websearch
pipe.connect(
    "websearch.documents", "prompt_builder_for_websearch.documents"
)
# 連接 prompt_builder_for_websearch 到 llm_for_websearch
pipe.connect(
    "prompt_builder_for_websearch", "llm_for_websearch"
)

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
  - router: ConditionalRouter
  - websearch: SerperDevWebSearch
  - prompt_builder_for_websearch: PromptBuilder
  - llm_for_websearch: OpenAIGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)
  - llm.replies -> router.replies (List[str])
  - router.go_to_websearch -> websearch.query (str)
  - router.go_to_websearch -> prompt_builder_for_websearch.query (str)
  - websearch.documents -> prompt_builder_for_websearch.documents (List[Document])
  - prompt_builder_for_websearch.prompt -> llm_for_websearch.prompt (str)

In [7]:
# 生成管道圖像
pipe.draw("pipe.png")

In [8]:
# 定義查詢
query = "慕尼黑在哪裡？"

# 運行管道
result = pipe.run({
    "prompt_builder": {
        "query": query,
        "documents": documents
    },
    "router": {"query": query}
})

# 輸出來自 ConditionalRouter 的 `answer`
print(result["router"]["answer"])
# ✅ 答案可以在定義的文件中找到。

慕尼黑是德國南部巴伐利亞州的首府。


In [9]:
# 定義查詢
query = "慕尼黑有多少人口？"

# 運行管道
result = pipe.run({
    "prompt_builder": {
        "query": query,
        "documents": documents
    },
    "router": {"query": query}
})

# 輸出使用網絡搜索文件生成的 `replies`
print(result["llm_for_websearch"]["replies"])

# 查看完整結果，你將看到網絡搜索組件還提供了從網上檢索到的文件鏈接：
print(result)

['根据搜集到的文档，截至 2020 年 7 月 31 日，慕尼黑的人口为 1,558,395 人。']
{'llm': {'meta': [{'model': 'gpt-4-turbo-2024-04-09', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 2, 'prompt_tokens': 557, 'total_tokens': 559}}]}, 'websearch': {'links': ['https://zh.wikipedia.org/zh-hans/%E6%85%95%E5%B0%BC%E9%BB%91', 'https://zh.wikipedia.org/zh-hans/%E5%BE%B7%E5%9B%BD%E5%9F%8E%E5%B8%82%E5%88%97%E8%A1%A8_(%E6%8C%89%E4%BA%BA%E5%8F%A3%E6%8E%92%E5%88%97)', 'https://baike.baidu.com/item/%E6%85%95%E5%B0%BC%E9%BB%91%E5%B8%82/12766218', 'https://www.wicco.net/city/149', 'http://goabroad.sohu.com/20090421/n263520876.shtml', 'https://www.tatsachen-ueber-deutschland.de/zh-hans/gailan/duocaizhiguo', 'https://yuanhang.buaa.edu.cn/yhwx/info/1240/4953.htm', 'http://fao.sz.gov.cn/ztzl/lszt/gjmcszft/content/post_11844.html', 'https://www.epochtimes.com/gb/11/11/29/n3443938.htm', 'https://www.aklishi.com/sjls/ozls/20438.html']}, 'llm_for_websearch': {'replies': ['根据搜集到的文档，截至 2020 年 7 月 31 日，慕尼黑的人口为 1,558,395